In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from las import LASReader

In [3]:
file = 'WELL-02_SONIC_LAS.txt'

In [4]:
data = LASReader(file, null_subs=np.nan)

#data.curves.items

#DT = data.data['DTC'][~np.isnan(data.data['DTC'])]


In [16]:
#df=pd.read_csv(file, skiprows=37, sep='\s+', na_values='-999.0000', header=None)
df = pd.DataFrame(data.data)

In [17]:
df.columns=['md','sonic','gr']

In [18]:
df['vi']=304800/df.sonic
df['sonic_metric'] = df.sonic * 3.2808

In [19]:
df = df.dropna()
# fillna(values around the missing)
df.head()

,md,sonic,gr,vi,sonic_metric
1709,310.4389,140.1644,25.3142,2174.589268,459.851364
1710,310.5913,164.4685,25.1973,1853.242414,539.588255
1711,310.7437,179.1637,24.4100,1701.237472,587.800267
1712,310.8961,154.6291,24.4300,1971.168428,507.307151
1713,311.0485,198.5657,26.0271,1535.008312,651.454349


In [20]:
data.start

49.9873

In [21]:
data.curves.items

{'DEPTH': LASItem(name='DEPTH', units='M', data='', descr='Depth'),
 'DTC': LASItem(name='DTC', units='us/ft', data='UNKNOWN', descr='DTC, run 1, version 1'),
 'GRC': LASItem(name='GRC', units='API', data='UNKNOWN', descr='GRC, run 1, version 1')}

## Replacement velocity input required

In [22]:
replacement_velocity = 1800
df['sonic_avg'] = (df.sonic_metric.shift(1) + df.sonic_metric) / 2000000
df.sonic_avg.iloc[0] = 1 / replacement_velocity # Replacement velocity
df.head()

,md,sonic,gr,vi,sonic_metric,sonic_avg
1709,310.4389,140.1644,25.3142,2174.589268,459.851364,0.000556
1710,310.5913,164.4685,25.1973,1853.242414,539.588255,0.000500
1711,310.7437,179.1637,24.4100,1701.237472,587.800267,0.000564
1712,310.8961,154.6291,24.4300,1971.168428,507.307151,0.000548
1713,311.0485,198.5657,26.0271,1535.008312,651.454349,0.000579


## One way time from checkshot to sonic value required

In [23]:
owt_from_checkshot = 0.15
df['owt_int'] = (df.md - df.md.shift(1)) * df.sonic_avg
df.owt_int.iloc[0] = owt_from_checkshot # one way time from checkshot to top of sonic log
df.head()

,md,sonic,gr,vi,sonic_metric,sonic_avg,owt_int
1709,310.4389,140.1644,25.3142,2174.589268,459.851364,0.000556,0.150000
1710,310.5913,164.4685,25.1973,1853.242414,539.588255,0.000500,0.000076
1711,310.7437,179.1637,24.4100,1701.237472,587.800267,0.000564,0.000086
1712,310.8961,154.6291,24.4300,1971.168428,507.307151,0.000548,0.000083
1713,311.0485,198.5657,26.0271,1535.008312,651.454349,0.000579,0.000088


In [24]:
df['owt_cum'] = df.owt_int.cumsum()
df.head()

,md,sonic,gr,vi,sonic_metric,sonic_avg,owt_int,owt_cum
1709,310.4389,140.1644,25.3142,2174.589268,459.851364,0.000556,0.150000,0.150000
1710,310.5913,164.4685,25.1973,1853.242414,539.588255,0.000500,0.000076,0.150076
1711,310.7437,179.1637,24.4100,1701.237472,587.800267,0.000564,0.000086,0.150162
1712,310.8961,154.6291,24.4300,1971.168428,507.307151,0.000548,0.000083,0.150246
1713,311.0485,198.5657,26.0271,1535.008312,651.454349,0.000579,0.000088,0.150334


## Plot owt_cum with checkshots

In [25]:
df['v_avg'] = df.md / df.owt_cum
#df.v_avg.iloc[0] = replacement_velocity
df.head()

,md,sonic,gr,vi,sonic_metric,sonic_avg,owt_int,owt_cum,v_avg
1709,310.4389,140.1644,25.3142,2174.589268,459.851364,0.000556,0.150000,0.150000,2069.592667
1710,310.5913,164.4685,25.1973,1853.242414,539.588255,0.000500,0.000076,0.150076,2069.557920
1711,310.7437,179.1637,24.4100,1701.237472,587.800267,0.000564,0.000086,0.150162,2069.388840
1712,310.8961,154.6291,24.4300,1971.168428,507.307151,0.000548,0.000083,0.150246,2069.253829
1713,311.0485,198.5657,26.0271,1535.008312,651.454349,0.000579,0.000088,0.150334,2069.052210


In [26]:
df['vi2_ti'] = np.nan

In [27]:
df.head()

,md,sonic,gr,vi,sonic_metric,sonic_avg,owt_int,owt_cum,v_avg,vi2_ti
1709,310.4389,140.1644,25.3142,2174.589268,459.851364,0.000556,0.150000,0.150000,2069.592667,NaN
1710,310.5913,164.4685,25.1973,1853.242414,539.588255,0.000500,0.000076,0.150076,2069.557920,NaN
1711,310.7437,179.1637,24.4100,1701.237472,587.800267,0.000564,0.000086,0.150162,2069.388840,NaN
1712,310.8961,154.6291,24.4300,1971.168428,507.307151,0.000548,0.000083,0.150246,2069.253829,NaN
1713,311.0485,198.5657,26.0271,1535.008312,651.454349,0.000579,0.000088,0.150334,2069.052210,NaN


In [33]:
df.vi2_ti.iloc[0] = replacement_velocity * df.owt_cum.iloc[0]
df.head()

,md,sonic,gr,vi,sonic_metric,sonic_avg,owt_int,owt_cum,v_avg,vi2_ti
1709,310.4389,140.1644,25.3142,2174.589268,459.851364,0.000556,0.150000,0.150000,2069.592667,270.0
1710,310.5913,164.4685,25.1973,1853.242414,539.588255,0.000500,0.000076,0.150076,2069.557920,NaN
1711,310.7437,179.1637,24.4100,1701.237472,587.800267,0.000564,0.000086,0.150162,2069.388840,NaN
1712,310.8961,154.6291,24.4300,1971.168428,507.307151,0.000548,0.000083,0.150246,2069.253829,NaN
1713,311.0485,198.5657,26.0271,1535.008312,651.454349,0.000579,0.000088,0.150334,2069.052210,NaN


In [34]:
df.vi2_ti.iloc[1:] = df.vi **2 * (df.owt_cum - df.owt_cum.shift(1))

df.head()

,md,sonic,gr,vi,sonic_metric,sonic_avg,owt_int,owt_cum,v_avg,vi2_ti
1709,310.4389,140.1644,25.3142,2174.589268,459.851364,0.000556,0.150000,0.150000,2069.592667,270.000000
1710,310.5913,164.4685,25.1973,1853.242414,539.588255,0.000500,0.000076,0.150076,2069.557920,261.562810
1711,310.7437,179.1637,24.4100,1701.237472,587.800267,0.000564,0.000086,0.150162,2069.388840,248.632822
1712,310.8961,154.6291,24.4300,1971.168428,507.307151,0.000548,0.000083,0.150246,2069.253829,324.234453
1713,311.0485,198.5657,26.0271,1535.008312,651.454349,0.000579,0.000088,0.150334,2069.052210,208.051328


In [35]:
df['v_rms'] = (df.vi2_ti.cumsum() / df.owt_cum) ** (0.5)
df.head()

,md,sonic,gr,vi,sonic_metric,sonic_avg,owt_int,owt_cum,v_avg,vi2_ti,v_rms
1709,310.4389,140.1644,25.3142,2174.589268,459.851364,0.000556,0.150000,0.150000,2069.592667,270.000000,42.426407
1710,310.5913,164.4685,25.1973,1853.242414,539.588255,0.000500,0.000076,0.150076,2069.557920,261.562810,59.514316
1711,310.7437,179.1637,24.4100,1701.237472,587.800267,0.000564,0.000086,0.150162,2069.388840,248.632822,72.081139
1712,310.8961,154.6291,24.4300,1971.168428,507.307151,0.000548,0.000083,0.150246,2069.253829,324.234453,85.737015
1713,311.0485,198.5657,26.0271,1535.008312,651.454349,0.000579,0.000088,0.150334,2069.052210,208.051328,93.436863


In [ ]:
reg = np.polyfit(df.md, df.vi, deg=1)
ry = np.polyval(reg, df.md)

plt.figure(figsize=(12,6))
plt.plot(df.md, df.vi,  'b', label='Velocity')
plt.plot(df.md, ry, 'r', label='regression')
plt.plot(df.md, df.v_avg, 'black')
#plt.plot(df.md, df.v_rms, 'green')
plt.legend(loc=0)
plt.grid(True)
plt.ylabel('Sonic velocity')
plt.xlabel('MD')
#plt.invert_yaxis()